# NOTES

First, you conducted a Hausman test to compare the fixed effects and random effects models. The Hausman test results suggest that the random effects model might be more appropriate. However, it's important to consider other factors, such as the economic theory and the specific characteristics of your data, when deciding between fixed and random effects.

To further investigate the model specification, you can try the following:

Lagged variables: Create lagged versions of your independent variables (e.g., by one year) to capture potential delayed effects.
Logarithmic transformations: Apply logarithmic transformations to variables with large values or skewed distributions to address potential non-linearity.
Interaction terms: Include interaction terms between independent variables to explore potential synergistic effects.
After each transformation, re-evaluate the model fit and compare the results. Additionally, make sure to check for potential issues like multicollinearity and heteroscedasticity.

The large numbers you are seeing might be due to scaling issues or the nature of the variables themselves. Consider standardizing or normalizing your variables to address scaling issues and make the coefficients more interpretable.

In [ ]:
# -*- coding: utf-8 -*-
"""script.ipynb

# IMPORTS AND SETTINGS
"""
!pip install linearmodels
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.api import add_constant
from linearmodels.panel import compare
import numpy as np
import scipy.stats
from linearmodels.panel import FirstDifferenceOLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import het_breuschpagan
import os
import warnings
warnings.filterwarnings('ignore')

# Set project folder
project_folder=os.getcwd()

# DATA READING
filename = os.path.join(project_folder, "Regression data", "panel_regression_data.csv")
data = pd.read_csv(panel_regression_data.csv)
data = data.set_index(['Country', 'Year'])
data = data.dropna(axis=1, how="all")
data.head(20)

# Logarithmic transformations
# Apply log transformation to variables with large values or skewed distributions
data["log_GDP_per_capita"] = np.log(data["GDP/capita"])
data["log_Tech_Advancement_CN"] = np.log(data["Tech Advancement CN"] + 1)  # Adding 1 to avoid log(0)
data["log_Fixed_Asset_Investment_CN_T1"] = np.log(data["Fixed Asset Investment CN (T-1)"] + 1)
data["log_Avg_Wage_Difference"] = np.log(data["Avg Wage Difference"] + 1)

# Lagged variables
# Create lagged versions of independent variables (by one year)
data["lag_GDP_per_capita"] = data.groupby(level=0)["GDP/capita"].shift(1)
data["lag_Tech_Advancement_CN"] = data.groupby(level=0)["Tech Advancement CN"].shift(1)
data["lag_Avg_Wage_Difference"] = data.groupby(level=0)["Avg Wage Difference"].shift(1)

# Interaction terms
# Create interaction terms between independent variables
data["GDP_Tech_Interaction"] = data["GDP/capita"] * data["Tech Advancement CN"]
data["Wage_Environ_Interaction"] = data["Avg Wage Difference"] * data["Environ. St. Difference"]

# Dropping any rows with missing values after transformation
data = data.dropna()

# Dependent, independent, and control variables
data["constant"] = 1
dependent_var = data['Import Share'].to_frame()

# Updated independent variables including lags, logs, and interactions
independent_vars = data[[
    "log_GDP_per_capita", "log_Tech_Advancement_CN", "log_Avg_Wage_Difference",
    "lag_GDP_per_capita", "lag_Tech_Advancement_CN", "lag_Avg_Wage_Difference",
    "GDP_Tech_Interaction", "Wage_Environ_Interaction"
]]

# Multicollinearity Check
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(independent_vars.values, i) for i in range(independent_vars.shape[1])]
vif["features"] = independent_vars.columns
print("Variance Inflation Factors (VIF):")
print(vif)

# Drop variables with high multicollinearity (VIF > 10 as a rule of thumb)
independent_vars = independent_vars.drop(columns=["GDP_Tech_Interaction"])  # Example drop, adjust as needed
independent_vars.columns

# MODELS
# Fixed Effects Model
fe_model = PanelOLS(dependent_var, independent_vars, entity_effects=True) # Added this code block for fe_results to be defined
fe_results = fe_model.fit()

# Random Effects Model (since Hausman suggests RE is more appropriate)
re_model = RandomEffects(dependent_var, independent_vars)
re_results = re_model.fit()

# IMPORTS AND SETTINGS

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.api import add_constant
from linearmodels.panel import compare
import numpy as np
import scipy.stats
from linearmodels.panel import FirstDifferenceOLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
# import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
from linearmodels.panel import PanelOLS
# from sklearn.preprocessing import StandardScaler
from statsmodels.stats.diagnostic import het_breuschpagan
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
project_folder=os.getcwd()
project_folder

'C:\\Users\\erott\\Documents\\Masterarbeit'

# SCRIPT

## DATA READING

'C:\\Users\\erott\\Documents\\Masterarbeit\\Regression data\\panel_regression_data.csv'

In [ ]:
# data from Excel
filename = os.path.join(project_folder, "Regression data", "panel_regression_data.csv")
data = pd.read_csv(os.path.join(project_folder, filename))
#'Country' as entity and 'Year' as time
data = data.set_index(['Country', 'Year'])
data=data.dropna(axis=1, how="all")
data.head(20)

Import Share    Fit  GDP/capita  Energy Consumption  \
Country Year                                                        
Austria 2005          0.86  0.672    38417.46              425.46   
        2006         15.73  0.678    40669.33              421.01   
        2007          7.33  0.521    46915.34              411.99   
        2008          6.67  0.559    51919.98              420.01   
        2009          6.99  0.530    48153.32              406.84   
        2010         17.15  0.434    46903.76              420.16   
        2011         18.52  0.438    51442.28              396.07   
        2012         28.38  0.338    48564.92              414.52   
        2013         19.75  0.230    50731.13              411.20   
        2014         16.42  0.000    51786.38              394.80   
        2015         12.38  0.000    44195.82              394.84   
        2016          7.06  0.000    45307.59              407.98   
        2017          6.28  0.000    47429.16              417.20   
        2018          8.37  0.000    51466.56              408.31   
        2019         19.79  0.000    50067.59              429.03   
Belgium 2005         16.27  0.000    36809.70              756.12   
        2006         40.78  0.000    38705.11              758.24   
        2007         33.90  0.000    44319.17              763.47   
        2008         38.33  0.000    48303.40              775.73   
        2009         35.59  0.000    44760.29              715.85   

              Annual Solar Capacity Addition  TechAdvancement   \
Country Year                                                     
Austria 2005                            -6.0               104   
        2006                             1.4               103   
        2007                             1.9               117   
        2008                             5.9               123   
        2009                            18.8                84   
        2010                            39.9                52   
        2011                            85.3                34   
        2012                           163.4                27   
        2013                           288.5                20   
        2014                           159.3                13   
        2015                           151.9                15   
        2016                           158.9                10   
        2017                           173.0                 7   
        2018                           186.2                11   
        2019                           247.0                 6   
Belgium 2005                             1.0                 3   
        2006                             0.0                 1   
        2007                            18.0                 3   
        2008                            42.0                 4   
        2009                           324.0                 4   

              Tech Advancement CN  Trade Policies EU  \
Country Year                                           
Austria 2005                  573               0.00   
        2006                  979               0.00   
        2007                 1621               0.00   
        2008                 2364               0.00   
        2009                 4149               0.00   
        2010                 6397               0.00   
        2011                 7634               0.00   
        2012                 8395               0.00   
        2013                 8057              21.84   
        2014                10136              47.70   
        2015                12974              47.70   
        2016                17340              47.70   
        2017                22199              47.70   
        2018                25048              33.79   
        2019                21697               0.00   
Belgium 2005                  573               0.00   
        2006                  979 

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 360 entries, ('Austria', 2005) to ('Sweden', 2019)
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Import Share                     360 non-null    float64
 1   Fit                              360 non-null    float64
 2   GDP/capita                       360 non-null    float64
 3   Energy Consumption               360 non-null    float64
 4   Annual Solar Capacity Addition   360 non-null    float64
 5   TechAdvancement                  360 non-null    int64  
 6   Tech Advancement CN              360 non-null    int64  
 7   Trade Policies EU                360 non-null    float64
 8   Fixed Asset Investment CN (T-1)  360 non-null    float64
 9   Avg Wage Difference              360 non-null    float64
 10  Environ. St. Difference          360 non-null    float64
dtypes: float64(9), int64(2)
memory usage: 33.1+ KB


In [ ]:
# #Standardisation of data
# scaler = StandardScaler()
# numerical_data = data.select_dtypes(include=['float64', 'int64'])
# scaled_data = scaler.fit_transform(numerical_data)
# data_scaled = pd.DataFrame(scaled_data, columns=numerical_data.columns, index=data.index)
# data_scaled

In [ ]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Import Share,360.0,18.471472,14.489463,0.2200,7.4125,15.5100,25.96750,88.2500
Fit,360.0,0.155722,0.227497,0.0000,0.0000,0.0000,0.22325,0.8300
GDP/capita,360.0,29856.376333,17016.864949,3899.8300,14699.5650,23613.7000,45540.14750,80848.3000
Energy Consumption,360.0,737.890194,943.847525,40.7500,187.9700,341.5450,835.88000,4062.3800
Annual Solar Capacity Addition,360.0,322.684167,1035.107133,-56.0000,0.2000,11.2500,141.47500,9539.0000
TechAdvancement,360.0,57.213889,153.733667,0.0000,3.0000,9.0000,35.25000,1242.0000
Tech Advancement CN,360.0,9970.866667,7896.538652,573.0000,2364.0000,8057.0000,17340.00000,25048.0000
Trade Policies EU,360.0,16.428667,21.118675,0.0000,0.0000,0.0000,47.70000,47.7000
Fixed Asset Investment CN (T-1),360.0,1053.947333,763.522139,63.5100,211.4700,1219.3800,1800.82000,2284.8700
Avg Wage Difference,360.0,1813.354492,1252.308905,-11.9100,775.5350,1416.5600,2904.22750,6615.0600


In [ ]:
#scaling of the data
data.loc[:,["GDP/capita", "Tech Advancement CN", "Fixed Asset Investment CN (T-1)", "Avg Wage Difference"]]=data.loc[:,["GDP/capita", "Tech Advancement CN", "Fixed Asset Investment CN (T-1)", "Avg Wage Difference"]].div(1000)
data.head(20)

Import Share    Fit  GDP/capita  Energy Consumption  \
Country Year                                                        
Austria 2005          0.86  0.672    38.41746              425.46   
        2006         15.73  0.678    40.66933              421.01   
        2007          7.33  0.521    46.91534              411.99   
        2008          6.67  0.559    51.91998              420.01   
        2009          6.99  0.530    48.15332              406.84   
        2010         17.15  0.434    46.90376              420.16   
        2011         18.52  0.438    51.44228              396.07   
        2012         28.38  0.338    48.56492              414.52   
        2013         19.75  0.230    50.73113              411.20   
        2014         16.42  0.000    51.78638              394.80   
        2015         12.38  0.000    44.19582              394.84   
        2016          7.06  0.000    45.30759              407.98   
        2017          6.28  0.000    47.42916              417.20   
        2018          8.37  0.000    51.46656              408.31   
        2019         19.79  0.000    50.06759              429.03   
Belgium 2005         16.27  0.000    36.80970              756.12   
        2006         40.78  0.000    38.70511              758.24   
        2007         33.90  0.000    44.31917              763.47   
        2008         38.33  0.000    48.30340              775.73   
        2009         35.59  0.000    44.76029              715.85   

              Annual Solar Capacity Addition  TechAdvancement   \
Country Year                                                     
Austria 2005                            -6.0               104   
        2006                             1.4               103   
        2007                             1.9               117   
        2008                             5.9               123   
        2009                            18.8                84   
        2010                            39.9                52   
        2011                            85.3                34   
        2012                           163.4                27   
        2013                           288.5                20   
        2014                           159.3                13   
        2015                           151.9                15   
        2016                           158.9                10   
        2017                           173.0                 7   
        2018                           186.2                11   
        2019                           247.0                 6   
Belgium 2005                             1.0                 3   
        2006                             0.0                 1   
        2007                            18.0                 3   
        2008                            42.0                 4   
        2009                           324.0                 4   

              Tech Advancement CN  Trade Policies EU  \
Country Year                                           
Austria 2005                0.573               0.00   
        2006                0.979               0.00   
        2007                1.621               0.00   
        2008                2.364               0.00   
        2009                4.149               0.00   
        2010                6.397               0.00   
        2011                7.634               0.00   
        2012                8.395               0.00   
        2013                8.057              21.84   
        2014               10.136              47.70   
        2015               12.974              47.70   
        2016               17.340              47.70   
        2017               22.199              47.70   
        2018               25.048              33.79   
        2019               21.697               0.00   
Belgium 2005                0.573               0.00   
        2006                0.979 

In [ ]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Import Share,360.0,18.471472,14.489463,0.22000,7.412500,15.51000,25.967500,88.25000
Fit,360.0,0.155722,0.227497,0.00000,0.000000,0.00000,0.223250,0.83000
GDP/capita,360.0,29.856376,17.016865,3.89983,14.699565,23.61370,45.540148,80.84830
Energy Consumption,360.0,737.890194,943.847525,40.75000,187.970000,341.54500,835.880000,4062.38000
Annual Solar Capacity Addition,360.0,322.684167,1035.107133,-56.00000,0.200000,11.25000,141.475000,9539.00000
TechAdvancement,360.0,57.213889,153.733667,0.00000,3.000000,9.00000,35.250000,1242.00000
Tech Advancement CN,360.0,9.970867,7.896539,0.57300,2.364000,8.05700,17.340000,25.04800
Trade Policies EU,360.0,16.428667,21.118675,0.00000,0.000000,0.00000,47.700000,47.70000
Fixed Asset Investment CN (T-1),360.0,1.053947,0.763522,0.06351,0.211470,1.21938,1.800820,2.28487
Avg Wage Difference,360.0,1.813354,1.252309,-0.01191,0.775535,1.41656,2.904228,6.61506


In [ ]:
# dependent, independent and control variables
data["constant"]=1
dependent_var = data['Import Share'].to_frame()
independent_vars = data.iloc[:,1:]
dependent_var.shape, independent_vars.shape

((360, 1), (360, 11))

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(independent_vars.values, i) for i in range(independent_vars.shape[1])]
vif["features"] = independent_vars.columns
print("Variance Inflation Factors (VIF):")
vif

Variance Inflation Factors (VIF):


,VIF Factor,features
0,1.315725,Fit
1,4.544590,GDP/capita
2,2.853626,Energy Consumption
3,2.384757,Annual Solar Capacity Addition
4,3.693410,TechAdvancement
5,8.957944,Tech Advancement CN
6,2.102597,Trade Policies EU
7,15.198968,Fixed Asset Investment CN (T-1)
8,5.154516,Avg Wage Difference
9,4.278517,Environ. St. Difference


In [ ]:
independent_vars = independent_vars.drop(columns=["constant","Fixed Asset Investment CN (T-1)"])
independent_vars.columns

Index(['Fit', 'GDP/capita', 'Energy Consumption',
       'Annual Solar Capacity Addition', 'TechAdvancement ',
       'Tech Advancement CN', 'Trade Policies EU', 'Avg Wage Difference',
       'Environ. St. Difference'],
      dtype='object')

## MODELS

### FE MODEL (ENTITY ONE WAY)

In [ ]:
# Fixed Effects Model
fe_model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
fe_results = fe_model.fit()

print("Fixed Effects Model Results:")
fe_results

Fixed Effects Model Results:


Dep. Variable:,Import Share,R-squared:,0.2203
Estimator:,PanelOLS,R-squared (Between):,-1.9870
No. Observations:,360,R-squared (Within):,0.2203
Date:,"Wed, May 15 2024",R-squared (Overall):,-1.3856
Time:,12:49:59,Log-likelihood,-1367.9
Cov. Estimator:,Unadjusted,,
,,F-statistic:,10.265
Entities:,24,P-value,0.0000
Avg Obs:,15.000,Distribution:,"F(9,327)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,10.265


### FE MODEL (TWO-WAY)

In [ ]:
#two-way fixed effects model
two_model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True,  drop_absorbed = True)
results = two_model.fit()
results

Dep. Variable:,Import Share,R-squared:,0.0924
Estimator:,PanelOLS,R-squared (Between):,-0.0249
No. Observations:,360,R-squared (Within):,-0.1615
Date:,"Wed, May 15 2024",R-squared (Overall):,-0.0621
Time:,12:50:01,Log-likelihood,-1342.1
Cov. Estimator:,Unadjusted,,
,,F-statistic:,4.5830
Entities:,24,P-value,0.0001
Avg Obs:,15.000,Distribution:,"F(7,315)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,4.5830


### RANDOM EFFECTS

In [ ]:
# Random Effects Model
re_model = RandomEffects(dependent_var, independent_vars)
re_results = re_model.fit()

print("\nRandom Effects Model Results:")
re_results


Random Effects Model Results:


Dep. Variable:,Import Share,R-squared:,0.4695
Estimator:,RandomEffects,R-squared (Between):,0.8808
No. Observations:,360,R-squared (Within):,0.1768
Date:,"Wed, May 15 2024",R-squared (Overall):,0.6890
Time:,12:50:02,Log-likelihood,-1395.4
Cov. Estimator:,Unadjusted,,
,,F-statistic:,34.512
Entities:,24,P-value,0.0000
Avg Obs:,15.000,Distribution:,"F(9,351)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,34.512


In [ ]:
#Hausmann Test
comparison = compare({"Fixed": fe_results, "Random": re_results})
comparison

,Fixed,Random
Dep. Variable,Import Share,Import Share
Estimator,PanelOLS,RandomEffects
No. Observations,360,360
Cov. Est.,Unadjusted,Unadjusted
R-squared,0.2203,0.4695
R-Squared (Within),0.2203,0.1768
R-Squared (Between),-1.9870,0.8808
R-Squared (Overall),-1.3856,0.6890
F-statistic,10.265,34.512
P-value (F-stat),0.0000,0.0000


### Durbin-Wu-Hausman test

In [ ]:
import numpy as np
from statsmodels.regression.linear_model import OLS
from scipy.stats import chi2

def durbin_wu_hausman_test(ols_fixed, ols_random):
    """
    Perform the Durbin-Wu-Hausman test for endogeneity.

    Parameters:
    ols_fixed : OLS
        Fixed-effects regression results
    ols_random : OLS
        Random-effects regression results

    Returns:
    chi_squared_stat : float
        The chi-squared test statistic
    p_value : float
        The p-value of the test
    """
    # Residuals from fixed effects model
    u_fe = ols_fixed.resids

    # Residuals from random effects model
    u_re = ols_random.resids

    # Calculate the test statistic
    dw_hausman_stat = np.sum((u_fe - u_re)**2) / np.sum(u_fe**2)

    df = fe_results.df_model - re_results.df_model

    # Compute the p-value
    p_value = 1 - chi2.cdf(dw_hausman_stat, df)

    return dw_hausman_stat, p_value

# Example usage:
# Assuming you have panel data stored in a DataFrame called 'data'
# with columns 'dependent_var', 'independent_var1', 'independent_var2', etc.
# and a variable 'id' for individual identifiers and 'time' for time identifiers.


# Perform the Durbin-Wu-Hausman test
dw_hausman_stat, p_value = durbin_wu_hausman_test(fe_results, re_results)

print("Durbin-Wu-Hausman Test Statistic:", dw_hausman_stat)
print("P-value:", p_value)

Durbin-Wu-Hausman Test Statistic: 0.16460656253201467
P-value: 1.0


In [ ]:
fe_results

Dep. Variable:,Import Share,R-squared:,0.2203
Estimator:,PanelOLS,R-squared (Between):,-1.9870
No. Observations:,360,R-squared (Within):,0.2203
Date:,"Tue, May 07 2024",R-squared (Overall):,-1.3856
Time:,18:06:06,Log-likelihood,-1367.9
Cov. Estimator:,Unadjusted,,
,,F-statistic:,10.265
Entities:,24,P-value,0.0000
Avg Obs:,15.000,Distribution:,"F(9,327)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,10.265


In [ ]:
#Wald Test: H0 is that all the regressors are not associated (no effect) with the dependent variables, H0 model has only the intercept (unconditional average)
wald_result = fe_results.wald_test(formula="Fit=0, `GDP/capita`=0, `Energy Consumption`=0, `Annual Solar Capacity Addition`=0, `Tech Advancement CN`=0, `TechAdvancement `=0, `Trade Policies EU`=0, `Avg Wage Difference`=0, `Environ. St. Difference`=0")
wald_result

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 92.3882
P-value: 0.0000
Distributed: chi2(9)
WaldTestStatistic, id: 0x1ce1dfef580

In [ ]:
# Perform the Breusch-Pagan test
residuals = fe_results.resids
independent_vars_bp_test = independent_vars.copy()
independent_vars_bp_test["constant"]=1
independent_vars_bp_test

bp_test = het_breuschpagan(resid=residuals, exog_het=independent_vars_bp_test, robust=True)
labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print(dict(zip(labels, bp_test)))

{'Lagrange multiplier statistic': 7.4883270442931815, 'p-value': 0.5864140706111666, 'f-value': 0.826108014941594, 'f p-value': 0.5924066635158942}
